In [11]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import gkey
gmaps.configure(api_key=gkey)

In [12]:
# Importing data from WeatherPy API request results
weather_info = pd.read_csv("..\WeatherPy\output_data\weather_data.csv")

In [13]:
weather_info.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Asău,46.4333,26.4000,17.51,48,0,1.08,RO,1655620480
1,Albany,42.6001,-73.9662,10.33,67,98,0.89,US,1655620345
2,Santa Rosa,14.3122,121.1114,34.39,56,63,1.08,PH,1655620484
3,Dikson,73.5069,80.5464,2.01,79,97,5.33,RU,1655620388
4,Nieuw Amsterdam,5.8833,-55.0833,25.01,89,79,3.71,SR,1655620486


In [14]:
# Setting up the data for mapping
locations = weather_info[["Lat", "Lng"]]
humidity = weather_info["Humidity"].astype(float)

In [15]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3.5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Wanting an artic experience: leaving only cold, cloudy and windy options

artic_options = weather_info[weather_info['Max Temp'] < 10]
artic_options = artic_options[artic_options['Cloudiness'] >= 80]
artic_options = artic_options[artic_options['Wind Speed'] > 6.5]
artic_options = artic_options.dropna()
artic_options

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,Tiksi,71.6872,128.8694,5.17,89,100,8.69,RU,1655620503
109,Saskylakh,71.9167,114.0833,6.81,56,100,10.59,RU,1655620632
155,Aklavik,68.2191,-135.0107,7.35,76,91,6.87,CA,1655620688
204,Klaksvík,62.2266,-6.5890,8.51,92,100,11.01,FO,1655620752
256,Waipawa,-41.4122,175.5153,8.03,80,100,6.94,NZ,1655620818
260,Örnsköldsvik,63.2909,18.7153,8.58,92,100,6.85,SE,1655620823
319,Rawson,-43.3002,-65.1023,5.60,72,98,6.76,AR,1655620849
343,Katangli,51.7102,143.2326,8.08,90,100,6.90,RU,1655620929
423,Tres Arroyos,-38.3739,-60.2798,8.78,37,100,7.35,AR,1655621036
579,Upernavik,72.7868,-56.1549,0.77,89,94,6.52,GL,1655621151


In [17]:
# Setting up the hotel dataframe
hotel_df = artic_options
hotel_df["Hotel Name"] = np.nan

In [18]:
# Setting parameters for Hotels within 5000 meters
params = {"types": "lodging", "radius": 5000, "key": gkey}

for index, row in hotel_df.iterrows():
    # Getting coords
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Making the API request
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        pass

In [19]:
# Showing the results
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
17,Tiksi,71.6872,128.8694,5.17,89,100,8.69,RU,1655620503,Arktika
109,Saskylakh,71.9167,114.0833,6.81,56,100,10.59,RU,1655620632,NaN
155,Aklavik,68.2191,-135.0107,7.35,76,91,6.87,CA,1655620688,Aklavik Inn
204,Klaksvík,62.2266,-6.5890,8.51,92,100,11.01,FO,1655620752,Airbnb rooms
256,Waipawa,-41.4122,175.5153,8.03,80,100,6.94,NZ,1655620818,Tora Coastal Walk
260,Örnsköldsvik,63.2909,18.7153,8.58,92,100,6.85,SE,1655620823,ファーストホテル スタット
319,Rawson,-43.3002,-65.1023,5.60,72,98,6.76,AR,1655620849,Hosteria Sampedro
343,Katangli,51.7102,143.2326,8.08,90,100,6.90,RU,1655620929,NaN
423,Tres Arroyos,-38.3739,-60.2798,8.78,37,100,7.35,AR,1655621036,Habitacion Triple con baño y entrada privada
579,Upernavik,72.7868,-56.1549,0.77,89,94,6.52,GL,1655621151,Café de Upernavik


In [20]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Setting up parameters
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

# Adding the layer to the map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations2, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))